In [1]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
bets = pd.read_csv('past_bets/base/bets_20220521.csv', index_col = 0)

In [3]:
bets = bets[bets.Home_Team != 'New York Yankees']
bets.reset_index(drop = True, inplace = True)
bets

,Home_Team,Away_Team,Home_Prob,Away_Prob,Home_ML,Away_ML,Home_ML_Prob,Away_ML_Prob,Home_Diff,Away_Diff,Home_KC,Away_KC,Home_Bet,Away_Bet
0,Chicago Cubs,Arizona Diamondbacks,0.581255,0.418745,-140.0,120.0,0.583333,0.454545,-0.002079,-0.035800,0.000000,0.000000,0.000000,0.000000
1,Toronto Blue Jays,Cincinnati Reds,0.772436,0.227564,-260.0,220.0,0.722222,0.312500,0.050213,-0.084936,0.018077,0.000000,1915.620554,0.000000
2,Colorado Rockies,New York Mets,0.473987,0.526013,110.0,-130.0,0.476190,0.565217,-0.002204,-0.039204,0.000000,0.000000,0.000000,0.000000
3,San Francisco Giants,San Diego Padres,0.601039,0.398961,-160.0,140.0,0.615385,0.416667,-0.014346,-0.017705,0.000000,0.000000,0.000000,0.000000
4,Boston Red Sox,Seattle Mariners,0.556904,0.443096,-155.0,135.0,0.607843,0.425532,-0.050939,0.017564,0.000000,0.003057,0.000000,324.004523
5,Philadelphia Phillies,Los Angeles Dodgers,0.537669,0.462331,-125.0,105.0,0.555556,0.487805,-0.017887,-0.025474,0.000000,0.000000,0.000000,0.000000
6,Cleveland Guardians,Detroit Tigers,0.692921,0.307079,-185.0,165.0,0.649123,0.377358,0.043798,-0.070280,0.012483,0.000000,1322.789233,0.000000
7,Miami Marlins,Atlanta Braves,0.429138,0.570862,125.0,-145.0,0.444444,0.591837,-0.015307,-0.020974,0.000000,0.000000,0.000000,0.000000
8,Pittsburgh Pirates,St. Louis Cardinals,0.437735,0.562265,120.0,-140.0,0.454545,0.583333,-0.016811,-0.021068,0.000000,0.000000,0.000000,0.000000
9,Baltimore Orioles,Tampa Bay Rays,0.352661,0.647339,120.0,-140.0,0.454545,0.583333,-0.101885,0.064006,0.000000,0.015361,0.000000,1627.870578


In [4]:
bets.to_csv('past_bets/base/bets_20220521.csv')